# Purspose

Extract bill numbers from a specific column in a lobbying dataset and compare these extracted bill numbers with another dataset to identify discrepancies. Specifically, it:

Reads the dataset containing lobbying information.

Extracts bill numbers from a text column using a regex pattern.

Joins the extracted bill numbers back with the original dataset.

Compares these extracted bill numbers with another dataset to find differences

Identifies and outputs discrepancies in bill numbers between the two datasets.

In [8]:
import pandas as pd
import re

# Read the CSV files
ldabills_df = pd.read_csv('lobbying_bills.csv')
ip4_bills = pd.read_csv('ip4_bill_links.csv')


In [9]:
# Sample data for demonstration
ldabills_df = pd.DataFrame({
    'report_year': [2022, 2022, 2023],
    'lobby_report_issue_id': [1, 2, 3],
    'specific_issue': [
        'Support for H.R. 1234 and opposition to S. 5678. Additional interest in H.R. 987.',
        'Monitoring S. 1122, H.R. 4321, and H.R. 8765 for potential impacts on industry.',
        'Advocacy for S. 3344.'
    ]
})

ip4_bills = pd.DataFrame({
    'lobby_report_issue_id': [1, 1, 2, 3],
    'bill_number': ['HR1234', 'S5678', 'HR4321', 'S3344']
})


In [10]:
# Function to extract bill numbers using regular expressions
def extract_bill_numbers(text):
    bill_pattern = r'(?<![a-z\d])\b(H\.?R|S)\.? ?\d+'  # Regex pattern to match bill numbers
    bills = re.findall(bill_pattern, text, flags=re.IGNORECASE) # Extract bill numbers
    bills = [re.sub(r'\s', '', bill) for bill in bills]  # Remove whitespace from bill numbers
    return bills

# Apply the function to the 'specific_issue' column
ldabills_df['extracted_bills'] = ldabills_df['specific_issue'].apply(extract_bill_numbers)
print(ldabills_df['extracted_bills'])



0    [H.R, S, H.R]
1    [S, H.R, H.R]
2              [S]
Name: extracted_bills, dtype: object


In [13]:
# Convert list to a dataframe and join with the original dataframe
bills_df = ldabills_df.explode('extracted_bills')[['report_year', 'lobby_report_issue_id', 'extracted_bills']]
print(bills_df)


   report_year  lobby_report_issue_id extracted_bills
0         2022                      1             H.R
0         2022                      1               S
0         2022                      1             H.R
1         2022                      2               S
1         2022                      2             H.R
1         2022                      2             H.R
2         2023                      3               S


In [14]:
# Normalize bill numbers by removing periods and spaces
ip4_bills['bill_number'] = ip4_bills['bill_number'].str.replace(r'\.| ', '', regex=True)
bills_df['extracted_bills'] = bills_df['extracted_bills'].str.replace(r'\.| ', '', regex=True)

In [15]:
# Combine bill identifier
ip4_bills['bill_identifier'] = (ip4_bills['lobby_report_issue_id'].astype(str) + "_" + ip4_bills['bill_number']).str.upper()
bills_df['bill_identifier'] = (bills_df['lobby_report_issue_id'].astype(str) + "_" + bills_df['extracted_bills']).str.upper()

print(ip4_bills)
print(bills_df)

   lobby_report_issue_id bill_number bill_identifier
0                      1      HR1234        1_HR1234
1                      1       S5678         1_S5678
2                      2      HR4321        2_HR4321
3                      3       S3344         3_S3344
   report_year  lobby_report_issue_id extracted_bills bill_identifier
0         2022                      1              HR            1_HR
0         2022                      1               S             1_S
0         2022                      1              HR            1_HR
1         2022                      2               S             2_S
1         2022                      2              HR            2_HR
1         2022                      2              HR            2_HR
2         2023                      3               S             3_S


In [16]:

# Bills in new extraction that are not in existing ip4 data
new_extraction_not_in_ip4 = bills_df[~bills_df['bill_identifier'].isin(ip4_bills['bill_identifier'])].groupby('report_year').size()


print(new_extraction_not_in_ip4)



report_year
2022    6
2023    1
dtype: int64


In [18]:
# Bills in existing ip4 data that is not in new extraction
ip4_not_in_new_extraction = ip4_bills[~ip4_bills['bill_identifier'].isin(bills_df['bill_identifier'])].groupby('report_year').size()

print(ip4_not_in_new_extraction)

KeyError: 'report_year'

In [ ]:

# Random examples from 2022
random_examples_2022 = bills_df[(~bills_df['bill_identifier'].isin(ip4_bills['bill_identifier'])) & (bills_df['report_year'] == 2022)].sample(10)



# View results
print(new_extraction_not_in_ip4)
print(ip4_not_in_new_extraction)
print(random_examples_2022)
